
Вам необходимо дописать функцию elasticity_df, которая принимает на вход датасет с предыдущего шага и возвращает эластичность для каждого SKU.

Чтобы подсчитать эластичность:

Для каждого товара постройте линейную зависимость логарифма продаж от цены.
Возьмите коэффициент детерминации линейной регрессии R 

 

Коэффициент детерминации показывает долю объясненной дисперсии в зависимости двух переменных. 

Если спрос товара хорошо объясняется ценой (есть явная зависимость), то коэффициент будет максимальный (неважно, какой, например, коэффициент наклона у кривой). Если объясняется плохо, коэффициент будет низкий, а на графике зависимости видно не будет.


Формат датасета:
```
     sku     dates   price  qty
0  10060  10/04/19  2798.0    3
1  10060  11/04/19  2798.0    2
2  10060  12/04/19  2798.0    4
3  10060  13/04/19  2798.0    3
4  10060  14/04/19  2798.0    1
5  10060  15/04/19  2821.0    4
6  10060  16/04/19  2821.0    4
7  10060  17/04/19  2821.0    4
8  10060  18/04/19  2821.0    3
9  10060  19/04/19  2821.0    2
```

Формула линейной зависимости спроса от цены
log(Q+1)=α∗P+β



In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

def elasticity_df(df: pd.DataFrame) -> pd.DataFrame:
    """Elasticity count"""
    #first
    df_g = df.groupby('sku')
    
    for sku, g in df_g:
        log_qty = np.log(df_g['qty'] + 1)
        price = df_g['price'].values.reshape(-1, 1)
        
        model = LinearRegression()
        model.fit(price, log_qty)
        
        log_qty_pred = model.predict(price)
        
        r_squared = r2_score(log_qty, log_qty_pred)
    
    return pd.DataFrame({'sku': df_g['sku'], 'elasticity' : r_squared})


In [10]:
# Пример данных
data = {
    'sku': [10060, 10060, 10060, 10060, 10060, 10060,10060,10060,10060,10060],
    'price': [2798.0, 2798.0, 2798.0, 2798.0, 2798.0, 2821.0, 2821.0,2821.0,2821.0,2821.0],
    'qty': [3,2,4,3,1,4,4,4,3,2]
}

# Создание DataFrame
df = pd.DataFrame(data)

def elasticity_df(df: pd.DataFrame) -> pd.DataFrame:
    grouped = df.groupby('sku')
    
    results = []
    
    for sku, group in grouped:
        # Логарифм продаж с добавлением 1 (чтобы избежать log(0))
        log_sales = np.log(group['qty'] + 1)
        price = group['price'].values.reshape(-1, 1)
        
        # Линейная регрессия
        model = LinearRegression()
        model.fit(price, log_sales)
        
        # Предсказанные значения логарифма продаж
        log_sales_pred = model.predict(price)
        
        # Коэффициент детерминации R^2
        r_squared = r2_score(log_sales, log_sales_pred)

        results.append({'sku': sku, 'elasticity': r_squared})
    
    results = pd.DataFrame(results)
    
    return results

print(elasticity_df(df))

     sku  elasticity
0  10060    0.156232


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import linregress


def linreg_rvalue(df: pd.DataFrame) -> float:
    p, q = df["price"], np.log1p(df["qty"])
    return linregress(p, q).rvalue ** 2


def elasticity_df(df: pd.DataFrame) -> pd.DataFrame:
    elast = df.groupby("sku")[["price", "qty"]].apply(linreg_rvalue)
    elast = elast.rename("elast")
    elast_df = pd.DataFrame()
    elast_df["sku"] = elast.index
    elast_df["elasticity"] = elast.values
    return elast_df